In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

# Load the match dataset
matches = pd.read_csv("matches.csv", index_col=0)

# Preprocessing
matches["date"] = pd.to_datetime(matches["date"])  # Convert string to datetime
matches["venue_code"] = matches["venue"].astype("category").cat.codes  # Encode venue
matches["opp_code"] = matches["opponent"].astype("category").cat.codes  # Encode opponent
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")  # Extract hour from time
matches["day_code"] = matches["date"].dt.dayofweek  # Extract day of the week
matches["target"] = (matches["result"] == "W").astype("int")  # 1 if win, else 0

# Define predictors
predictors = ["venue_code", "opp_code", "hour", "day_code"]

# Split the data into training and testing sets
train = matches[matches["date"] < "2025-01-01"]
test = matches[matches["date"] > "2025-01-01"]

# Train Random Forest model
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
rf.fit(train[predictors], train["target"])

# Make predictions and evaluate
preds = rf.predict(test[predictors])
acc = accuracy_score(test["target"], preds)
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
print("Accuracy:", acc)
print("Precision:", precision_score(test["target"], preds))

# Calculate rolling averages for performance-based features
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed="left").mean()
    group[new_cols] = rolling_stats
    return group.dropna(subset=new_cols)

# Define performance features
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

# Apply rolling averages per team
matches_rollings = matches.groupby("team", group_keys=False).apply(
    lambda x: rolling_averages(x, cols, new_cols)
)

# Reset index after groupby
matches_rollings.index = range(matches_rollings.shape[0])

# Re-train the model with performance stats
def make_predictions(data, predictors):
    train = data[data["date"] < "2025-01-01"]
    test = data[data["date"] > "2025-01-01"]
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

# Make predictions using updated dataset
combined, precision = make_predictions(matches_rollings, predictors + new_cols)
combined = combined.merge(matches_rollings[["date", "team", "opponent", "result"]],
                          left_index=True, right_index=True)

# Map long team names to short names using custom dictionary
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

mapping = MissingDict(**map_values)
combined["new_team"] = combined["team"].map(mapping)

# Merge data to compare predictions of both teams in a match
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

# Evaluate accuracy of mutual predictions (team x predicted win, team y predicted loss)
print(merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts())


In [ ]:
# SUMMARY:
# 1. Loaded and cleaned match data.
# 2. Created features for model training including venue, time, opponent, and performance-based rolling stats.
# 3. Trained a Random Forest model to predict win/loss.
# 4. Evaluated prediction quality using accuracy and precision.
# 5. Merged predictions to simulate matches between teams.
# 6. Set up structure to compare mutual predictions in head-to-head matches.
